In [2]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [48]:
data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_9'
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source1 + '/'
else:
    _dir = data_source1 + '\\'

clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]

In [49]:
#get cluster wise data for summarzation
count = 0
count_not = 0
count_yes = 0
precentage = 0
score = []
for cluster in clusters:
    df = pd.read_csv(cluster + '/1385_LR_bellwether_g.csv')
    counts = []
    for row in range(df.shape[0]):
        j = df.iloc[row].values[1:]
        value = sum(i <= 0.33 for i in j)
        counts.append(value)
    df['>=0.6'] = counts
    precentage = max(counts)/df.shape[0]
    #print(cluster,df.iloc[counts.index(max(counts)),0])
    score.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.33),max(counts),round(np.mean(counts),2),df.iloc[counts.index(max(counts)),0]])
    count += max(counts)
score_df = pd.DataFrame(score, columns = ['id','expected','max_bell','avge','bellwether'])
score_df = score_df.sort_values('id')
score_df.to_csv('data/1385/projects/bellwether.csv')
#print(score_df)

In [50]:
#get cluster wise data for summarzation using median
count = 0
count_not = 0
count_yes = 0
precentage = 0
score = []
score_med = []
cluster_info = {}
for cluster in clusters:
    df = pd.read_csv(cluster + '/1385_LR_bellwether_f1.csv')
    counts = []
    med_count = []
    for row in range(df.shape[0]):
        j = df.iloc[row].values[1:]
        j_med = np.median(j)
        value = sum(i >= 0.66 for i in j)
        med_count.append(j_med)
        counts.append(value)
    df['>=0.6'] = counts
    precentage = max(counts)/df.shape[0]
    x = int(cluster.rsplit('/',1)[1])
    cluster_info[x] = df.iloc[med_count.index(min(med_count)),0]
    #print(cluster,df.iloc[counts.index(max(counts)),0])
    score.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.33),max(counts),round(np.mean(counts),2),df.iloc[counts.index(max(counts)),0]])
    score_med.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.33),counts[med_count.index(max(med_count))],round(np.mean(counts),2),max(med_count),df.iloc[med_count.index(max(med_count)),0]])
    count += max(counts)
score_df = pd.DataFrame(score_med, columns = ['id','expected','max_bell','avge','median','bellwether'])
score_df = score_df.sort_values('id')
score_df.to_csv('data/1385/projects/bellwether.csv')

In [51]:
print(score_df.expected.sum(),score_df.max_bell.sum())

202 198


In [26]:
od = collections.OrderedDict(sorted(cluster_info.items()))
d = dict(od)

In [27]:
#get cluster wise data for summarzation using median at upper level
count = 0
count_not = 0
count_yes = 0
precentage = 0
score = []
score_med = []
for cluster in clusters:
    df = pd.read_csv(cluster + '/1385_LR_bellwether_f1.csv')
    x_count = 0
    if int(cluster.rsplit('/',1)[1]) not in cluster_info.keys():
        counts = []
        med_count = []
        for row in range(df.shape[0]):
            j = df.iloc[row].values[1:]
            j_med = np.median(j)
            value = sum(i >= 0.66 for i in j)
            med_count.append(j_med)
            counts.append(value)
        df['>=0.6'] = counts
        #print(int(cluster.rsplit('/',1)[1]),med_count)
        precentage = max(counts)/df.shape[0]
        score_med.append([int(cluster.rsplit('/',1)[1]),
                          round(df.shape[0]*0.33),
                          counts[med_count.index(max(med_count))],
                          round(np.mean(counts),2),max(med_count),
                          df.iloc[med_count.index(max(med_count)),0]])
        count += max(counts)
    else:
        counts = []
        med_count = []
        projects = []
        for row in range(df.shape[0]):
            if df.iloc[row].values[0] not in cluster_info[int(cluster.rsplit('/',1)[1])].values():
                continue
            projects.append(df.iloc[row].values[0])
            x_count += 1
            j = df.iloc[row].values[1:]
            j_med = np.median(j)
            value = sum(i >= 0.66 for i in j)
            med_count.append(j_med)
            counts.append(value)
        #print(int(cluster.rsplit('/',1)[1]),med_count,counts[med_count.index(max(med_count))],counts,projects)
        precentage = max(counts)/df.shape[0]
        score_med.append([int(cluster.rsplit('/',1)[1]),
                          round(df.shape[0]*0.33),
                          counts[med_count.index(max(med_count))],
                          round(np.mean(counts),2),max(med_count),
                          projects[med_count.index(max(med_count))]])
score_df = pd.DataFrame(score_med, columns = ['id','expected','max_bell','avge','median','bellwether'])
score_df = score_df.sort_values('id')
score_df.to_csv('data/1385/projects/bellwether.csv')

AttributeError: 'str' object has no attribute 'values'

In [523]:
score_df

,id,expected,max_bell,avge,median,bellwether
4,1,29,33,22.22,0.550,benojt.csv
2,11,1,1,0.50,0.350,columba.csv
7,14,33,34,19.00,0.560,jmule.csv
8,25,1,1,0.50,0.605,magicwars.csv
5,26,1,0,0.50,0.575,turbotrader-bos.csv
1,27,0,0,0.00,0.610,xmemcached.csv
3,28,21,21,14.17,0.525,botsnscouts.csv
0,35,73,80,62.21,0.590,libarchive.csv
6,55,67,67,47.05,0.520,grisbi.csv


In [521]:
#f1
cluster_info = {1:{2: 'logicmail.csv',
 3: 'benojt.csv',
 4: 'exist.csv',
 5: 'springside.csv',
 6: 'geotools.csv',
 7: 'joustsim.csv',
 8: 'javaprofiler.csv',
 9: 'jgnash.csv',
 10: 'mycila.csv'},
 14:{15: 'amanda.csv',
 16: 'cascading.csv',
 17: 'wxcrp.csv',
 18: 'enlightenment.csv',
 19: 'jmule.csv',
 20: 'lmms.csv',
 21: 'superwaba.csv',
 22: 'voms.csv',
 23: 'simplewebservices.csv',
 24: 'patterntesting.csv'},
 28:{29: 'bochs.csv',
 30: 'boost-sandbox.csv',
 31: 'jython.csv',
 32: 'google-caja.csv',
 33: 'botsnscouts.csv',
 34: 'poormans.csv'},
 35:{36: 'igarden.csv',
 37: 'mob-sen-plat.csv',
 38: 'jajuk.csv',
 39: 'libarchive.csv',
 40: 'sbml.csv',
 41: 'mcmc-jags.csv',
 42: 'zengine.csv',
 43: 'kowari.csv',
 44: 'qgo.csv',
 45: 'electric-core.csv',
 46: 'irrlicht.csv',
 47: 'reactos-mirror.csv',
 48: 'jparsec.csv',
 49: 'pcgen.csv',
 50: 'sumatrapdf.csv',
 51: 'mscore.csv',
 52: 'elmo.csv',
 53: 'mcore3d.csv',
 54: 'mimosa.csv'},
 55:{56: 'ivataopenportal.csv',
 57: 'fontforge.csv',
 58: 'graphmak.csv',
 59: 'nh3d.csv',
 60: 'optalgtoolkit.csv',
 61: 'xuggle.csv',
 62: 'encog-cs.csv',
 63: 'jmol.csv',
 64: 'nug.csv',
 65: 'dvt.csv',
 66: 'plplot.csv',
 67: 'openintents.csv',
 68: 'silvertree.csv',
 69: 'jrdf.csv',
 70: 'wpdev.csv',
 71: 'grisbi.csv',
 72: 'liferea.csv',
 73: 'kolmafia.csv',
 74: 'rebecca-aiml.csv',
 75: 'anaxagora.csv'}}

In [506]:
#Precision
cluster_info = {1:{2: 'mclient-mume.csv',
 3: 'jatlas.csv',
 4: 'icescrum.csv',
 5: 'springside.csv',
 6: 'gabel.csv',
 7: 'jgossipforum.csv',
 8: 'jmoney.csv',
 9: 'mp-rechnungs-und-kundenverwaltung.csv',
 10: 'mycila.csv'},
 14:{15: 'amanda.csv',
 16: 'cascading.csv',
 17: 'aufs.csv',
 18: 'jfreereport.csv',
 19: 'ng4j.csv',
 20: 'lmms.csv',
 21: 'amateur-scrolls.csv',
 22: 'fm-classic.csv',
 23: 'gdis.csv',
 24: 'jaxlib.csv'},
 28:{29: 'h2database.csv',
 30: 'nodal.csv',
 31: 'opentibia.csv',
 32: 'myfaces.csv',
 33: 'freenet.csv',
 34: 'poormans.csv'},
 35: {
 36: 'quark.csv',
 37: 'm-a-d-n-e-s-s.csv',
 38: 'jajuk.csv',
 39: 'customwars.csv',
 40: 'reaper-ecad.csv',
 41: 'pidgin.csv',
 42: 'zildo.csv',
 43: 'rtb-team.csv',
 44: 'applet2app.csv',
 45: 'adaptagrams.csv',
 46: 'zoolib.csv',
 47: 'etics.csv',
 48: 'rezilla.csv',
 49: 'tolven.csv',
 50: 'modellus.csv',
 51: 'all-plasm.csv',
 52: 'sigil.csv',
 53: 'projecteqemu.csv',
 54: 'google-web-toolkit.csv'},
 55: {56: 'mvdsv.csv',
 57: 'mia.csv',
 58: 'pdfedit.csv',
 59: 'jaxen.csv',
 60: 'zact.csv',
 61: 'opengs.csv',
 62: 'opennms.csv',
 63: 'papertoolkit.csv',
 64: 'nug.csv',
 65: 'codepurge.csv',
 66: 'plplot.csv',
 67: 'alembik.csv',
 68: 'silvertree.csv',
 69: 'jrdf.csv',
 70: 'java-gnome.csv',
 71: 'proftp.csv',
 72: 'liferea.csv',
 73: 'elvishrayrenderer.csv',
 74: 'moving-pictures.csv',
 75: 'anaxagora.csv'}}

In [518]:
#Recall
cluster_info = {1:{2: 'logicmail.csv',
 3: 'benojt.csv',
 4: 'exist.csv',
 5: 'springside.csv',
 6: 'google-collections.csv',
 7: 'ngl.csv',
 8: 'javaprofiler.csv',
 9: 'yarp0.csv',
 10: 'mycila.csv'},
 14:{
 15: 'amanda.csv',
 16: 'unflobtactical.csv',
 17: 'gebr.csv',
 18: 'enlightenment.csv',
 19: 'bl-toolkit.csv',
 20: 'lmms.csv',
 21: 'multitude.csv',
 22: 'voms.csv',
 23: 'kml.csv',
 24: 'werx.csv'},
 28:{
 29: 'galaxium.csv',
 30: 'smbopensoft.csv',
 31: 'somap.csv',
 32: 'fleet-simulator.csv',
 33: 'sqlpower-library.csv',
 34: 'uface.csv'},
 35:{
 36: 'pnotepad.csv',
 37: 'jguard.csv',
 38: 'jajuk.csv',
 39: 'mpgravity.csv',
 40: 'planeshift.csv',
 41: 'libtorrent.csv',
 42: 'prelude.csv',
 43: 'apolloplayer.csv',
 44: 'hydrogen.csv',
 45: 'electric-core.csv',
 46: 'irrlicht.csv',
 47: 'critterding.csv1',
 48: 'jparsec.csv',
 49: 'pcgen.csv',
 50: 'sumatrapdf.csv',
 51: 'virtuoso.csv',
 52: 'sylpheed-claws.csv',
 53: 'mcore3d.csv',
 54: 'mimosa.csv'},
 55:{
 56: 'ivataopenportal.csv',
 57: 'tcl.csv',
 58: 'rectracker.csv',
 59: 'nh3d.csv',
 60: 'abbot.csv',
 61: 'scite-ru.csv',
 62: 'sevenupdate.csv',
 63: 'jmol.csv',
 64: 'nug.csv',
 65: 'ldview.csv',
 66: 'buddata-ebxml-registry.csv',
 67: 'barracudamvc.csv',
 68: 'silvertree.csv',
 69: 'jrdf.csv',
 70: 'wpdev.csv',
 71: 'grisbi.csv',
 72: 'ipfilter.csv',
 73: 'megasquirtavr.csv',
 74: 'glelite.csv',
 75: 'jsr308-langtools.csv'}}

In [510]:
#g-score
cluster_info = {1:{2: 'mclient-mume.csv',
 3: 'chrysalis.csv',
 4: 'opendvr.csv',
 5: 'springside.csv',
 6: 'twostep.csv',
 7: 'sblim.csv',
 8: 'matrex.csv',
 9: 'yarp0.csv',
 10: 'mycila.csv'},
 14:{
 15: 'amanda.csv',
 16: 'cascading.csv',
 17: 'google-gears.csv',
 18: 'pcsx2.csv',
 19: 'jmule.csv',
 20: 'qpe.csv',
 21: 'fritzing.csv',
 22: 'voms.csv',
 23: 'ppcboot.csv',
 24: 'exl.csv'},
 28:{
 29: 'bochs.csv',
 30: 'boost-sandbox.csv',
 31: 'kdenlive.csv',
 32: 'triplea.csv',
 33: 'botsnscouts.csv',
 34: 'uface.csv'},
 35:{
 36: 'xcsoar.csv',
 37: 'jguard.csv',
 38: 'jajuk.csv',
 39: 'gss.csv',
 40: 'maya-work-in-progress.csv',
 41: 'natpad.csv',
 42: 'owlib.csv',
 43: 'apolloplayer.csv',
 44: 'lphant.csv',
 45: 'olsrd.csv',
 46: 'auber.csv',
 47: 'nativeclient.csv',
 48: 'webdriver.csv',
 49: 'selenium.csv',
 50: 'sumatrapdf.csv',
 51: 'all-plasm.csv',
 52: 'neoengine.csv',
 53: 'projecteqemu.csv',
 54: 'ftm.csv'},
 55:{
 56: 'olex2.csv',
 57: 'jcl.csv',
 58: 'graphmak.csv',
 59: 'jaxen.csv',
 60: 'abbot.csv',
 61: 'frogatto.csv',
 62: 'intact.csv',
 63: 'papertoolkit.csv',
 64: 'oorexx.csv',
 65: 'free-evolution.csv',
 66: 'lcfizvtop.csv',
 67: 'irplus.csv',
 68: 'freewrl.csv',
 69: 'jrdf.csv',
 70: 'unicase.csv',
 71: 'mesa3d.csv',
 72: 'opensim4opencog.csv',
 73: 'xharbour.csv',
 74: 'rebecca-aiml.csv',
 75: 'htmlunit.csv'}}

In [513]:
#pf
cluster_info = {1:{2: 'mclient-mume.csv',
 3: 'jatlas.csv',
 4: 'sidekar.csv',
 5: 'calcubetimer.csv',
 6: 'avisynth2.csv',
 7: 'jbpm.csv',
 8: 'jchassis.csv',
 9: 'openefm.csv',
 10: 'octet.csv'},
 14:{
 15: 'empyrean.csv',
 16: 'cascading.csv',
 17: 'jnode.csv',
 18: 'ffigo.csv',
 19: 'aztec.csv',
 20: 'lmms.csv',
 21: 'massiv.csv',
 22: 'foursquared.csv',
 23: 'wow-qrsk.csv',
 24: 'zealos.csv'},
 28:{
 29: 'bzbyte.csv',
 30: 'mecat.csv',
 31: 'cme.csv',
 32: 'fbmanager.csv',
 33: 'mediaportal.csv',
 34: 'fido.csv'},
 35:{
 36: 'levent.csv',
 37: 'm-a-d-n-e-s-s.csv',
 38: 'zorannt.csv',
 39: 'growl-for-windows.csv',
 40: 'reaper-ecad.csv',
 41: 'xtreemfs.csv',
 42: 'fluidium.csv',
 43: 'xreal.csv',
 44: 'xined.csv',
 45: 'badtrinitycore.csv',
 46: 'genoviz.csv',
 47: 'monetdb.csv',
 48: 'rezilla.csv',
 49: 'tolven.csv',
 50: 'jstock.csv',
 51: 'nosleepsoftware.csv',
 52: 'omseek.csv',
 53: 'xbplayer.csv',
 54: 'autat.csv'},
 55:{
 56: 'mvdsv.csv',
 57: 'quantlib.csv',
 58: 'graphmak.csv',
 59: 'jvcl.csv',
 60: 'jicarilla.csv',
 61: 'linuxsh.csv',
 62: 'opennms.csv',
 63: 'papertoolkit.csv',
 64: 'nassp.csv',
 65: 'uclmda.csv',
 66: 'mvnforum.csv',
 67: 'mevenide.csv',
 68: 'freewrl.csv',
 69: 'jrdf.csv',
 70: 'java-gnome.csv',
 71: 'tcvp.csv',
 72: 'thrust.csv',
 73: 'jruby.csv',
 74: 'aubit4gl.csv',
 75: 'gtad.csv'}}

In [524]:
bellwethers = ['benojt.csv','jmule.csv','botsnscouts.csv',
               'libarchive.csv','grisbi.csv','magicwars.csv',
               'turbotrader-bos.csv','xmemcached.csv','columba.csv',
               'enlightenment.csv','sqlpower-library.csv','hydrogen.csv',
               'nh3d.csv','mp-rechnungs-und-kundenverwaltung.csv','h2database.csv',
               'reaper-ecad.csv','alembik.csv','personalaccess.csv','emftriple.csv',
               'jchassis.csv','empyrean.csv','fbmanager.csv','growl-for-windows.csv',
               'quantlib.csv','javagroups.csv','twostep.csv','nativeclient.csv',
               'rebecca-aiml.csv','turbotrader-bos.csv','gpsmid.csv']

In [525]:
len(bellwethers)

30

In [527]:
with open('data/1385/exp1/1385_cluster_0.pkl', 'rb') as handle:
    _cluster_projects = pickle.load(handle)